In [3]:
import pandas as pd
from experiment_utils.env_setup import init
from experiment_utils.utils import FileHandler
from pathlib import Path


2024-08-25 16:30:18 - INFO - PyTorch version 2.2.2 available.


In [4]:
from arabert import ArabertPreprocessor
from transformers import AutoTokenizer

In [5]:
model_path = 'aubmindlab/bert-base-arabertv02'
tokenizer = AutoTokenizer.from_pretrained(model_path)
preprocessor = ArabertPreprocessor(model_path)

/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
variant = "conll2003_bert"
base_folder = init()
experiment_folder = base_folder / "Final Year Experiments/Thesis-Experiments/Experiments/DashboardTest"
variant_folder = experiment_folder / variant

2024-08-25 16:30:20 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-08-25 16:30:20 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [7]:
file_name = '/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/test_ignored.json'
test_data = pd.read_json(
	# "/Users/ay227/Desktop/Final-Year/Deformer-Development/test_ignored.json",
	file_name,
 	lines=True
)

In [9]:
extractions_fh = FileHandler(variant_folder / 'extractions')
analysis_data = extractions_fh.read_json("analysis_data.json")
kmeans_results = extractions_fh.read_json("kmeans_results.json")




In [10]:
kmeans_results


,n_clusters,homogeneity,completeness,v_measure
0,"{'0': 'k=3', '1': 'k=4', '2': 'k=9'}","{'0': 0.7349820369000001, '1': 0.7067036725, '...","{'0': 0.7192699071, '1': 0.7384163046000001, '...","{'0': 0.7270410930000001, '1': 0.7222120274, '..."


In [15]:
df = analysis_data[analysis_data['labels'] !=-100].copy()


In [16]:
df.columns

Index(['sentence_ids', 'token_positions', 'words', 'tokens', 'word_pieces',
       'core_tokens', 'true_labels', 'token_selector_id', 'pred_labels',
       'agreements', 'x', 'y', 'labels', 'losses', 'token_ids', 'global_id',
       'true_token_score', 'pred_token_score', 'k=3', 'boundary_clusters',
       'k=4', 'entity_clusters', 'k=9', 'token_clusters', 'consistency_count',
       'inconsistency_count', 'total_train_occurrences', 'local_token_entropy',
       'token_max_entropy', 'dataset_token_entropy', 'local_word_entropy',
       'word_max_entropy', 'dataset_word_entropy', 'tokenization_rate',
       'tr_entity', 'pr_entity', 'error_type', 'O', 'B-PERS', 'I-PERS',
       'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC',
       'prediction_entropy', 'prediction_max_entropy', 'confidence',
       'variability', 'pre_x', 'pre_y'],
      dtype='object')

In [17]:
import json
file_path = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/DashboardTest/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
with open(file_path, 'r') as f:
    data = json.load(f)


In [24]:
import pandas as pd
token_report  = pd.DataFrame(data['token_report']).rename(columns={'Tag':'true_labels'})

In [25]:
token_report

,true_labels,Precision,Recall,F1,Support
0,B-LOC,0.9004267,0.9476048,0.9234136,668
1,B-MISC,0.8131313,0.6851064,0.7436490,235
2,B-ORG,0.8629442,0.7555556,0.8056872,450
3,B-PERS,0.8835856,0.8846154,0.8841002,858
4,I-LOC,0.8117647,0.8313253,0.8214286,83
5,I-MISC,0.8658537,0.4303030,0.5748988,165
6,I-ORG,0.8312236,0.7163636,0.7695312,275
7,I-PERS,0.9089506,0.9188768,0.9138867,641
8,O,0.9856323,0.9933383,0.9894703,21616
9,accuracy/micro,0.9719899,0.9719899,0.9719899,24991


In [74]:
df['consistency_ratio'] = df['consistency_count'] / df['total_train_occurrences']
df['inconsistency_ratio'] = df['inconsistency_count'] / df['total_train_occurrences']
df['prediction_entropy_ratio'] = df['prediction_entropy'] / df['prediction_max_entropy']
df['local_token_entropy_ratio'] = df['local_token_entropy'] / df['token_max_entropy']
df['local_word_entropy_ratio'] = df['local_word_entropy'] / df['word_max_entropy']

In [75]:
df['local_token_entropy'].describe()

count    24991.000000
mean         0.004915
std          0.461034
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.008257
max          2.501609
Name: local_token_entropy, dtype: float64

In [108]:
aggregated_df = df.groupby('true_labels')[
    ['consistency_ratio', 'inconsistency_ratio', 'total_train_occurrences',
    'prediction_entropy_ratio', 'prediction_entropy', 'local_token_entropy',
    'local_token_entropy_ratio', 'tokenization_rate',
    'local_word_entropy', 'local_word_entropy_ratio', 'confidence', 'variability']
    ].agg('mean').reset_index()


In [109]:
df.columns

Index(['sentence_ids', 'token_positions', 'words', 'tokens', 'word_pieces',
       'core_tokens', 'true_labels', 'token_selector_id', 'pred_labels',
       'agreements', 'x', 'y', 'labels', 'losses', 'token_ids', 'global_id',
       'true_token_score', 'pred_token_score', 'k=3', 'boundary_clusters',
       'k=4', 'entity_clusters', 'k=9', 'token_clusters', 'consistency_count',
       'inconsistency_count', 'total_train_occurrences', 'local_token_entropy',
       'token_max_entropy', 'dataset_token_entropy', 'local_word_entropy',
       'word_max_entropy', 'dataset_word_entropy', 'tokenization_rate',
       'tr_entity', 'pr_entity', 'error_type', 'O', 'B-PERS', 'I-PERS',
       'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC',
       'prediction_entropy', 'prediction_max_entropy', 'confidence',
       'variability', 'pre_x', 'pre_y', 'consistency_ratio',
       'inconsistency_ratio', 'prediction_entropy_ratio',
       'local_token_entropy_ratio', 'local_word_entropy_ratio'],
     

In [110]:
correlation_df = pd.merge(aggregated_df, token_report, on="true_labels", how="inner").set_index('true_labels')
correlation_cols = [col for col in  correlation_df.columns if col not in ['total_train_occurrences', 'Support']]

In [111]:
correlation_cols

['consistency_ratio',
 'inconsistency_ratio',
 'prediction_entropy_ratio',
 'prediction_entropy',
 'local_token_entropy',
 'local_token_entropy_ratio',
 'tokenization_rate',
 'local_word_entropy',
 'local_word_entropy_ratio',
 'confidence',
 'variability',
 'Precision',
 'Recall',
 'F1']

In [166]:
import plotly.express as px
import numpy as np
pearson_matrix = correlation_df[correlation_cols].corr(method="pearson")
spearman_matrix = correlation_df[correlation_cols].corr(method="spearman")
# correlation_matrix = pearson_matrix - spearman_matrix
correlation_matrix = pearson_matrix
# Step 2: Mask the upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Create a DataFrame for the masked correlation matrix to plot with NaNs in the upper triangle
masked_correlation_matrix = correlation_matrix.mask(mask)

correlation_fig = px.imshow(
                masked_correlation_matrix,
                labels=dict(x="Variables", y="Variables", color="Correlation"),
                title='Correlation Matrix of Aggregated Data',
                width=800,  # Custom width
                height=600  # Custom height
            )

In [170]:
df.columns

Index(['sentence_ids', 'token_positions', 'words', 'tokens', 'word_pieces',
       'core_tokens', 'true_labels', 'token_selector_id', 'pred_labels',
       'agreements', 'x', 'y', 'labels', 'losses', 'token_ids', 'global_id',
       'true_token_score', 'pred_token_score', 'k=3', 'boundary_clusters',
       'k=4', 'entity_clusters', 'k=9', 'token_clusters', 'consistency_count',
       'inconsistency_count', 'total_train_occurrences', 'local_token_entropy',
       'token_max_entropy', 'dataset_token_entropy', 'local_word_entropy',
       'word_max_entropy', 'dataset_word_entropy', 'tokenization_rate',
       'tr_entity', 'pr_entity', 'error_type', 'O', 'B-PERS', 'I-PERS',
       'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC',
       'prediction_entropy', 'prediction_max_entropy', 'confidence',
       'variability', 'pre_x', 'pre_y', 'consistency_ratio',
       'inconsistency_ratio', 'prediction_entropy_ratio',
       'local_token_entropy_ratio', 'local_word_entropy_ratio'],
     

In [167]:
correlation_matrix

,consistency_ratio,inconsistency_ratio,prediction_entropy_ratio,prediction_entropy,local_token_entropy,local_token_entropy_ratio,tokenization_rate,local_word_entropy,local_word_entropy_ratio,confidence,variability,Precision,Recall,F1
consistency_ratio,1.000000,-1.000000,-0.747488,-0.747488,-0.644342,-0.609702,-0.290407,-0.324005,-0.594669,0.704040,0.711970,0.672008,0.791880,0.821404
inconsistency_ratio,-1.000000,1.000000,0.747488,0.747488,0.644342,0.609702,0.290407,0.324005,0.594669,-0.704040,-0.711970,-0.672008,-0.791880,-0.821404
prediction_entropy_ratio,-0.747488,0.747488,1.000000,1.000000,0.211014,0.354754,0.023054,0.098497,0.372620,-0.993946,-0.995800,-0.596721,-0.988181,-0.983720
prediction_entropy,-0.747488,0.747488,1.000000,1.000000,0.211014,0.354754,0.023054,0.098497,0.372620,-0.993946,-0.995800,-0.596721,-0.988181,-0.983720
local_token_entropy,-0.644342,0.644342,0.211014,0.211014,1.000000,0.227426,-0.062712,0.873009,0.136846,-0.118414,-0.133291,-0.651893,-0.266067,-0.346917
local_token_entropy_ratio,-0.609702,0.609702,0.354754,0.354754,0.227426,1.000000,0.703463,-0.156625,0.982187,-0.326175,-0.326485,-0.614405,-0.317041,-0.353518
tokenization_rate,-0.290407,0.290407,0.023054,0.023054,-0.062712,0.703463,1.000000,-0.511619,0.801838,-0.014218,-0.017742,-0.003276,-0.012872,-0.002872
local_word_entropy,-0.324005,0.324005,0.098497,0.098497,0.873009,-0.156625,-0.511619,1.000000,-0.279277,-0.017226,-0.028995,-0.552834,-0.137972,-0.213877
local_word_entropy_ratio,-0.594669,0.594669,0.372620,0.372620,0.136846,0.982187,0.801838,-0.279277,1.000000,-0.352192,-0.352737,-0.510843,-0.339892,-0.360456
confidence,0.704040,-0.704040,-0.993946,-0.993946,-0.118414,-0.326175,-0.014218,-0.017226,-0.352192,1.000000,0.999726,0.531309,0.982779,0.969577


In [168]:
 -0.017226 - - 0.300000

0.28277399999999997

In [169]:
correlation_fig

In [94]:
correlation_fig

In [116]:
correlation_fig

In [165]:
fig = px.scatter(
    correlation_df.reset_index(),
    x="local_token_entropy_ratio",
    y="inconsistency_ratio",
    color="true_labels",  # Use 'Metric' to color the data points
    # hover_data=["Metric", "Value"],  # Display additional info on hover
    title='Scatter Plot of Various Metrics by Label',
    
)

# # Customize the layout for better visualization
# fig.update_layout(
#     xaxis_title="Label",
#     yaxis_title="Metric Value",
#     template='plotly_white',  # Use a white background for better clarity
#     plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
#     paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent paper background
#     legend_title_text='Metric'  # Legend title
# )

# Show the plot
fig.show()

In [81]:
import numpy as np
def normalized_entropy(df, col1, col2, new_col_name):
    return np.where(
        (df[col1] == 0) & (df[col2] == 0),  # Condition for both columns being 0
        0,  # Value if condition is true
        np.where(
            (df[col1] == -1) & (df[col2] == -1),  # Condition for both columns being -1
            -1,  # Value if condition is true
            df[col1] / df[col2]  # Default calculation
        )
    )
    

In [82]:
normalized_entropy(df, 'local_token_entropy', 'token_max_entropy', 'normalized')

array([-1.       ,  0.       ,  0.       , ...,  0.2502454,  1.       ,
        0.       ])

In [83]:
df[['local_token_entropy', 'token_max_entropy', 'Normalized', 'confidence', 'B-LOC', 'O']]

,local_token_entropy,token_max_entropy,Normalized,confidence,B-LOC,O
1,-1.000000,-1.000000,1.000000,0.762098,0.762098,0.219626
2,0.000000,0.000000,0.000000,0.988486,0.988486,0.006432
3,0.000000,0.000000,0.000000,0.999737,0.000075,0.999737
4,-1.000000,-1.000000,1.000000,0.977785,0.001534,0.009206
5,0.591673,1.000000,0.591673,0.990327,0.000692,0.001699
...,...,...,...,...,...,...
29704,0.000000,0.000000,0.000000,0.999930,0.000004,0.999930
29705,0.503258,1.000000,0.503258,0.975661,0.002424,0.019219
29707,0.581052,2.321928,0.250245,0.995962,0.000115,0.000830
29708,1.000000,1.000000,1.000000,0.994623,0.000334,0.001225


In [53]:
df[TagAmbiguityColumns.TRUE_LABELS]

KeyError: 'True Labels'

In [55]:
df.columns

Index(['sentence_ids', 'token_positions', 'words', 'tokens', 'word_pieces',
       'core_tokens', 'true_labels', 'token_selector_id', 'pred_labels',
       'agreements', 'x', 'y', 'labels', 'losses', 'token_ids', 'global_id',
       'true_token_score', 'pred_token_score', 'k=3', 'boundary_clusters',
       'k=4', 'entity_clusters', 'k=9', 'token_clusters', 'consistency_count',
       'inconsistency_count', 'total_train_occurrences', 'local_token_entropy',
       'token_max_entropy', 'dataset_token_entropy', 'local_word_entropy',
       'word_max_entropy', 'dataset_word_entropy', 'tokenization_rate',
       'tr_entity', 'pr_entity', 'error_type', 'O', 'B-PERS', 'I-PERS',
       'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC',
       'prediction_entropy', 'prediction_max_entropy', 'confidence',
       'variability', 'pre_x', 'pre_y'],
      dtype='object')

In [47]:
from dataclasses import dataclass

@dataclass(frozen=True)
class TagAmbiguityColumns:
    TRUE_LABELS: str = 'true_labels'
    CORE_TOKENS: str = 'Core Tokens'
    MEAN_CONSISTENCY: str = 'Consistency Count'
    MEAN_INCONSISTENCY: str = 'Inconsistency Count'
    MEAN_TOKEN_ENTROPY: str = 'Local Token Entropy'
    OVERALL_MEAN_CONSISTENCY: str = 'overall_mean_consistency'
    OVERALL_MEAN_INCONSISTENCY: str = 'overall_mean_inconsistency'
    OVERALL_MEAN_TOKEN_ENTROPY: str = 'overall_mean_token_entropy'

def create_tag_ambiguity_table(selected_df):
        columns = TagAmbiguityColumns()
        print(selected_df[TagAmbiguityColumns.MEAN_CONSISTENCY].dtype)
        print(selected_df[TagAmbiguityColumns.MEAN_INCONSISTENCY].dtype)
        print(selected_df[TagAmbiguityColumns.MEAN_TOKEN_ENTROPY].dtype)
        print(selected_df[[TagAmbiguityColumns.MEAN_CONSISTENCY, TagAmbiguityColumns.MEAN_INCONSISTENCY, TagAmbiguityColumns.MEAN_TOKEN_ENTROPY]].head())

        tag_ambiguity_analysis = selected_df.groupby([columns.TRUE_LABELS, columns.CORE_TOKENS]).agg(
            mean_consistency=(columns.MEAN_CONSISTENCY, 'mean')
        ).reset_index()

        print(tag_ambiguity_analysis.head())
        
        

        print(selected_df[TagAmbiguityColumns.MEAN_CONSISTENCY])
        # Perform the initial group by and aggregation
        tag_ambiguity_analysis = selected_df.groupby([columns.TRUE_LABELS, columns.CORE_TOKENS]).agg(
            mean_consistency=(columns.MEAN_CONSISTENCY, 'mean'),
            mean_inconsistency=(columns.MEAN_INCONSISTENCY, 'mean'),
            mean_token_entropy=(columns.MEAN_TOKEN_ENTROPY, 'mean')
        ).reset_index()

        # Further aggregation to summarize the results
        tag_ambiguity_summary = tag_ambiguity_analysis.groupby(columns.TRUE_LABELS).agg(
            overall_mean_consistency=(columns.MEAN_CONSISTENCY, 'mean'),
            overall_mean_inconsistency=(columns.MEAN_INCONSISTENCY, 'mean'),
            overall_mean_token_entropy=(columns.MEAN_TOKEN_ENTROPY, 'mean')
        ).reset_index().round(3)

        # Use the CustomDataTable class to render the DataTable
        return tag_ambiguity_summary



In [48]:
create_tag_ambiguity_table(df)

KeyError: 'Consistency Count'

In [28]:
df[df['normalized_token_entropy'].isna()]

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y,normalized_token_entropy
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,9.502024e-04,0.000260,0.000119,0.112901,3.169925,0.988486,0.310205,2.727193,7.541053,NaN
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,1.974570e-05,0.000017,0.000020,0.004256,3.169925,0.999737,0.314177,3.418482,4.236978,NaN
8,0,8,-,-,[-],-,O,-@#8@#0,O,True,...,1.084640e-05,0.000014,0.000013,0.002388,3.169925,0.999860,0.314220,3.433832,4.270978,NaN
9,0,9,أمر,أمر,[أمر],أمر,O,أمر@#9@#0,O,True,...,1.392200e-06,0.000005,0.000003,0.000601,3.169925,0.999969,0.314259,-2.095126,12.691721,NaN
10,0,10,جلالة,جلالة,[جلالة],جلالة,O,جلالة@#10@#0,O,True,...,2.408700e-06,0.000010,0.000005,0.001669,3.169925,0.999905,0.314236,3.612740,9.714819,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29698,960,2,صدر,صدر,[صدر],صدر,O,صدر@#2@#960,O,True,...,9.308000e-07,0.000009,0.000002,0.000580,3.169925,0.999970,0.314259,-0.525271,14.372940,NaN
29699,960,3,مؤخراً,مؤخرا,[مؤخرا],مؤخرا,O,مؤخرا@#3@#960,O,True,...,1.703100e-06,0.000015,0.000004,0.000881,3.169925,0.999953,0.314253,-1.266616,14.762239,NaN
29702,960,6,عن,عن,[عن],عن,O,عن@#6@#960,O,True,...,9.959000e-07,0.000010,0.000007,0.000652,3.169925,0.999966,0.314258,-2.634144,14.662696,NaN
29704,960,8,الشخصية,الشخصية,[الشخصية],الشخصية,O,الشخصية@#8@#960,O,True,...,5.060000e-06,0.000019,0.000006,0.001265,3.169925,0.999930,0.314245,1.589292,12.428691,NaN


In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class CustomColumns:
    GROUND_TRUTH: str = 'Ground Truth'
    ANCHOR_TOKEN: str = 'Anchor Token'
    MEAN_CONSISTENCY: str = 'mean_consistency'
    MEAN_INCONSISTENCY: str = 'mean_inconsistency'
    MEAN_TOKEN_ENTROPY: str = 'mean_token_entropy'
    OVERALL_MEAN_CONSISTENCY: str = 'overall_mean_consistency'
    OVERALL_MEAN_INCONSISTENCY: str = 'overall_mean_inconsistency'
    OVERALL_MEAN_TOKEN_ENTROPY: str = 'overall_mean_token_entropy'


In [2]:
c = CustomColumns()
   


In [ ]:
c.

In [11]:
analysis_data[analysis_data['losses'] <0]

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y


In [11]:
analysis_data[analysis_data['labels'] !=-100]

KeyError: 'labels'

In [8]:
analysis_data[analysis_data['labels'] != -100]

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
1,0,1,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,...,0.921583,0.001045,0.002406,0.000270,0.479903,3.169925,0.921583,0.287163,-0.383466,6.988535
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,0.976076,0.003101,0.000382,0.000405,0.211169,3.169925,0.976076,0.305834,3.833407,8.589263
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,0.000043,0.000017,0.000011,0.000020,0.004157,3.169925,0.999741,0.314178,-2.705303,4.609430
4,0,4,غيث,غيث,[غيث],غيث,B-PERS,غيث@#4@#0,B-PERS,True,...,0.005369,0.001407,0.003879,0.001247,0.459931,3.169925,0.941825,0.293780,3.725152,8.169139
5,0,5,الطراونة,الطر,"[الطر, ##او, ##نة]",الطر,I-PERS,الطر@#5@#0,I-PERS,True,...,0.001115,0.000397,0.000584,0.000948,0.182628,3.169925,0.980874,0.307515,3.628871,7.993730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29704,960,8,الشخصية,الشخصية,[الشخصية],الشخصية,O,الشخصية@#8@#960,O,True,...,0.000004,0.000002,0.000008,0.000007,0.000730,3.169925,0.999961,0.314256,2.324433,12.266322
29705,960,9,للوليد,للو,"[للو, ##ليد]",للو,B-PERS,للو@#9@#960,B-PERS,True,...,0.003278,0.000800,0.005411,0.001723,0.805358,3.169925,0.818223,0.255096,3.745475,9.541816
29707,960,11,بن,بن,[بن],بن,I-PERS,بن@#11@#960,I-PERS,True,...,0.000210,0.000174,0.000154,0.000345,0.075092,3.169925,0.993027,0.311806,3.822868,9.497227
29708,960,12,طلال,طلال,[طلال],طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,0.000228,0.000216,0.000095,0.000284,0.078278,3.169925,0.992675,0.311681,4.015800,9.701574


In [9]:
analysis_data[~analysis_data['true_token_score'].isna()]

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
1,0,1,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,...,0.921583,0.001045,0.002406,0.000270,0.479903,3.169925,0.921583,0.287163,-0.383466,6.988535
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,0.976076,0.003101,0.000382,0.000405,0.211169,3.169925,0.976076,0.305834,3.833407,8.589263
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,0.000043,0.000017,0.000011,0.000020,0.004157,3.169925,0.999741,0.314178,-2.705303,4.609430
4,0,4,غيث,غيث,[غيث],غيث,B-PERS,غيث@#4@#0,B-PERS,True,...,0.005369,0.001407,0.003879,0.001247,0.459931,3.169925,0.941825,0.293780,3.725152,8.169139
5,0,5,الطراونة,الطر,"[الطر, ##او, ##نة]",الطر,I-PERS,الطر@#5@#0,I-PERS,True,...,0.001115,0.000397,0.000584,0.000948,0.182628,3.169925,0.980874,0.307515,3.628871,7.993730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29704,960,8,الشخصية,الشخصية,[الشخصية],الشخصية,O,الشخصية@#8@#960,O,True,...,0.000004,0.000002,0.000008,0.000007,0.000730,3.169925,0.999961,0.314256,2.324433,12.266322
29705,960,9,للوليد,للو,"[للو, ##ليد]",للو,B-PERS,للو@#9@#960,B-PERS,True,...,0.003278,0.000800,0.005411,0.001723,0.805358,3.169925,0.818223,0.255096,3.745475,9.541816
29707,960,11,بن,بن,[بن],بن,I-PERS,بن@#11@#960,I-PERS,True,...,0.000210,0.000174,0.000154,0.000345,0.075092,3.169925,0.993027,0.311806,3.822868,9.497227
29708,960,12,طلال,طلال,[طلال],طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,0.000228,0.000216,0.000095,0.000284,0.078278,3.169925,0.992675,0.311681,4.015800,9.701574


In [10]:
analysis_data['true_labels'].value_counts()

true_labels
O          21633
IGNORED     2781
[CLS]        961
[SEP]        961
B-PERS       858
B-LOC        668
I-PERS       641
B-ORG        450
I-ORG        275
B-MISC       235
I-MISC       165
I-LOC         83
Name: count, dtype: int64

In [11]:
len(analysis_data) - (2781+961+961)

25008

In [25]:
df = extractions_fh.read_json("kmeans_results.json")
df.columns

Index(['n_clusters', 'homogeneity', 'completeness', 'v_measure'], dtype='object')

In [15]:
results.columns

Index(['Precision', 'Recall', 'F1', 'Loss', 'Type', 'true_score',
       'pred_score'],
      dtype='object')

In [10]:
config_fh = FileHandler("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase")
config  = config_fh.load_yaml("dashboard-config.yaml")

In [17]:
len(config.get('dashboard').get('data').get('analysis_data').get('column_mappings'))

52